# 7. Sequential GridSearchCV

In [1]:
import joblib
import os

import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.layers import Dense
from keras.callbacks import EarlyStopping

from scikeras.wrappers import KerasRegressor

import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
%run ./scripts/store_model_performance.py
%run ./scripts/atomic_benchmark_estimator.py
%run ./scripts/get_all_possible_combinations.py

In [3]:
df = pd.read_pickle("./pickles/003.dataframe.data-preprocessing.pkl")
df

,price,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bedrooms,review_scores_rating,...,Shampoo,Smoke alarm,Stove,TV,Washer,Wifi,Entire place,Hotel room,Private room,Shared room
0,53,0,0.000138,1,0,0.543185,0.012963,0.066667,0.0,1.0,...,0,0,0,0,1,1,1.0,0.0,0.0,0.0
1,120,0,0.000138,1,1,0.543180,0.013029,0.066667,0.0,1.0,...,1,0,0,0,1,1,1.0,0.0,0.0,0.0
2,89,0,0.000138,1,0,0.543124,0.012873,0.066667,0.0,1.0,...,0,0,0,1,1,1,1.0,0.0,0.0,0.0
3,58,0,0.000138,1,1,0.542730,0.012810,0.066667,0.0,1.0,...,0,0,0,1,0,1,1.0,0.0,0.0,0.0
4,60,0,0.000138,1,0,0.542833,0.012610,0.066667,0.0,1.0,...,0,0,0,1,1,1,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279707,120,0,0.000138,1,1,0.542522,0.012857,0.066667,0.0,1.0,...,0,1,0,1,1,1,1.0,0.0,0.0,0.0
279708,60,0,0.000138,1,1,0.543257,0.012956,0.066667,0.0,1.0,...,1,0,0,1,1,1,1.0,0.0,0.0,0.0
279709,50,0,0.000138,1,1,0.543189,0.013051,0.066667,0.0,1.0,...,1,0,0,0,1,1,1.0,0.0,0.0,0.0
279710,105,0,0.000138,1,1,0.542965,0.013229,0.066667,0.0,1.0,...,1,1,0,1,1,1,1.0,0.0,0.0,0.0


## 7.1. GridSearch

In [4]:
input_size = df.shape[1] - 1
input_size

63

In [6]:
def create_model(layers, model_opt, first_activation, first_n_nodes):
    model = keras.models.Sequential()

    model.add(Dense(first_n_nodes, input_shape=(input_size,), activation=first_activation))

    for act, n_nodes in zip(*layers):
        model.add(Dense(n_nodes, activation=act))

    model.add(Dense(1, activation="linear"))

    model.compile(
        optimizer=model_opt,
        loss="mae",
    )
    
    return model

In [6]:
activations = ["tanh"]

In [7]:
n_nodes = [10, 64, 64 * 2]

In [8]:
pd.DataFrame(r_get_all_combinations(2, [activations, n_nodes]))

,0,1
0,"[tanh, tanh]","[10, 10]"
1,"[tanh, tanh]","[10, 64]"
2,"[tanh, tanh]","[10, 128]"
3,"[tanh, tanh]","[64, 10]"
4,"[tanh, tanh]","[64, 64]"
5,"[tanh, tanh]","[64, 128]"
6,"[tanh, tanh]","[128, 10]"
7,"[tanh, tanh]","[128, 64]"
8,"[tanh, tanh]","[128, 128]"


In [9]:
list(range(5, 13, 3))

[5, 8, 11]

In [10]:
params = {
    "layers": r_get_all_combinations(2, [activations, n_nodes]), # layers
    "model_opt": ["adam"],
    "epochs": list(range(10, 60, 20)),
    "first_activation": activations,
    "first_n_nodes": n_nodes,
    "batch_size": [10],
}

In [11]:
np.product([len(p) for p in params.values()]) * 6 # CV

486

In [13]:
early_stopping_monitor = EarlyStopping(patience=2, monitor="loss")
early_stopping_monitor

In [14]:
keras_regressor = KerasRegressor(
    model=create_model,
    verbose=3,
    callbacks=[early_stopping_monitor],
    **{n: params[n][0] for n in params},
)

In [15]:
grid = GridSearchCV(
    estimator=keras_regressor,
    param_grid=params,
    n_jobs=15,
    cv=6,
    scoring="neg_mean_absolute_error",
    verbose=3,
    error_score="raise",
)

In [16]:
df.iloc[:, 1:].to_numpy()

array([[0.00000000e+00, 1.38217001e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.38217001e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.38217001e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 1.38217001e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.38217001e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.38217001e-04, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [17]:
df.iloc[:, :1].to_numpy()

array([[53],
       [120],
       [89],
       ...,
       [50],
       [105],
       [70]], dtype=object)

In [ ]:
grid.fit(df.iloc[:, 1:].to_numpy(), df.iloc[:, :1].to_numpy(), verbose=3)

Fitting 6 folds for each of 81 candidates, totalling 486 fits


C:\Users\lande\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Because the grid searchcv took too long, it was stopped while 'training'. The log output is stored in a [separate file](data/007.gridsearchcv-output.log)
We will extract the output [here](008.grid-search-cv-extract.ipynb)

In [ ]:
grid.best_params_